In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F



class Model(nn.Module):
    def __init__(pretrained = false):
        super(Model, self).__init__()
        if (pretrained == true):
            self.model = torchvision.models.vgg16(pretrained=True)
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.fc = nn.Linear(128, 1) 
            else:
                self.model = nn.Sequential(
                    nn.Conv2d(3, 16, 5), 
                    ,nn.BatchNorm2d(16)
                        ,F.relu()
                        ,nn.Conv2d(16, 32, 5)
                        ,nn.BatchNorm2d(32)
                            ,F.relu()
                         ,nn.Conv2d(32, 128, 5)
                            ,nn.BatchNorm2d(128)
                            ,F.relu()
                            ,nn.Conv2d(128, 256, 5)
                                ,nn.BatchNorm2d(256)
                                ,nn.Linear(256, 128)
                                    ,nn.BatchNorm1d(128)
                                    ,F.relu()
                                    ,nn.Linear(128, 1)
                ) 
    
    def forward(self, input_data):
        return self.model(input_data)
class Solver():
    def __init__:
        super(Solver, self).__init__()
        
    def loss(self, pred, labels):
        return loss = nn.MSELoss(pred, labels)

    def train(self, input_data, output, num_epochs = 10, learning_rate = 1e-2, batch_size = 128, pretrained = false):

            train_loader = torch.utils.data.DataLoader(dataset = input_data,

                                               batch_size=batch_size, 

                                               shuffle=True)

            cnn = Model(pretrained)
            optimizer = torch.optim.Adam(cnn.parameters, lr = learning_rate)

            for i in range(num_epochs):
                for j, (images, labels) in enumerate(training_loader):
                    images = Variable(images)
                    labels = Variable(labels)
                    optimizer.zero_grad()
                    pred = self.model(input_data)
                    loss = self.loss(pred, output)
                    net.zero_grad()
                    loss.backward()
                    optimizer.step()

                    if (i+1) % 100 == 0:

                        print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 

                       %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

solver = Solver()
solver.train()         
    
        
    
    
    
    